# Using Spectral Weights

This guide shows how to weight Scans and spectra during averaging and how to find out what weights were used in the process.


You can find a copy of this tutorial as a Jupyter notebook [here](https://github.com/GreenBankObservatory/dysh/blob/main/notebooks/examples/using_spectral_weights.ipynb) or download it by right clicking  <a href="https://raw.githubusercontent.com/GreenBankObservatory/dysh/refs/heads/main/notebooks/examples/using_spectral_weights.ipynb" download>here</a> and selecting "Save Link As".

## Loading Modules
We start by loading the modules we will use for the data reduction. 

For display purposes, we use the static (non-interactive) matplotlib backend in this tutorial. However, you can tell `matplotlib` to use the `ipympl` backend to enable interactive plots. This is only needed if working on jupyter lab or notebook.

In [ ]:
# Set interactive plots in jupyter.
#%matplotlib ipympl

# These modules are required for this example.
import numpy as np
import astropy.units as u
from dysh.spectra import tsys_weight
from dysh.spectra import average_spectra
from dysh.fits.gbtfitsload import GBTFITSLoad

# These modules are only used to download the data.
from pathlib import Path
from dysh.util.download import from_url

## Data Retrieval

Download the example SDFITS data, if necessary.

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/positionswitch/data/AGBT05B_047_01/AGBT05B_047_01.raw.acs/AGBT05B_047_01.raw.acs.fits"
savepath = Path.cwd() / "data"
savepath.mkdir(exist_ok=True) # Create the data directory if it does not exist.
filename = from_url(url, savepath)

## Data Loading

Next, we use [`GBTFITSLoad`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.fits.html#dysh.fits.gbtfitsload.GBTFITSLoad) to load the data, and then its [`summary`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.fits.html#dysh.fits.gbtfitsload.GBTFITSLoad.summary) method to inspect its contents.

In [ ]:
sdf = GBTFITSLoad(filename)
sdf.summary()

### Position Switched Calibration
We calibrate several scans at once for NGC5291.
This results in a [`ScanBlock`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock) with 4 scans, each with 11 integrations. 
The default weights are unity, with one weight value per integration.

In [ ]:
pssb = sdf.getps(object='NGC5291', ifnum=0, plnum=1, fdnum=0)
print(f"Number of scans: {len(pssb)}, Number of integrations per scan: {[k.nint for k in pssb]}")
print(f"Scan weights: {pssb.weights}")

## Applying weights when averaging Scans
When time averaging integrations and/or scans to create a final spectrum, you can choose from dysh's two options ('tsys' or None) or supply your own weight array.

### 1. System temperature weighting
The default for [`timeaverage()`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock.timeaverage) is 'tsys' which calculates and applies system temperature weighting to each integration: 

$w = t_{exp} \times \delta\nu/T_{sys}^2$,

where $t_{exp}$ is the integration exposure time, $\delta\nu$ is the channel width, and $T_{sys}$ is the system temperature.
You can check what those weights will be with [`tsys_weight`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.SpectralAverageMixin.tsys_weight). 

In [ ]:
pssb.tsys_weight

(Note this is an array of arrays rather than a multidimensional array because different scans may have differing numbers of integrations).

[`timeaverage()`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock.timeaverage) with the default arguments will use these weights.
The final spectral weight is the sum of all weights.

In the following lines of code we will time average the $4\times11$ integrations using the default weights ('tsys'), and then remove a baseline from the resulting time average. For the baseline removal we exclude ranges at the edges of the spectrum and where we know there's a signal. We end by plotting the time averged baseline subtracted spectrum, and printing the weights and statistics.

In [ ]:
ta1 = pssb.timeaverage()  # default is weights='tsys'

# Define regions to be excluded from the baseline fit.
exclude_regions = [(1.37*u.GHz,1.38*u.GHz),
                   (1.395*u.GHz,1.405*u.GHz),
                   (1.42*u.GHz,1.43*u.GHz)]

# Fit an order 1 polynomial excluding the ranges defined above and subtract it (remove=True).
ta1.baseline(degree=1, remove=True, exclude=exclude_regions)

# Plot the result.
ta1.plot(ymin=-0.25, ymax=0.3)

# Print final weights and statistics.
print(f"final weights={ta1.weights}")
ta1.stats()

### 2. Equal weighting
Supplying `weights=None` will weight all integrations the same.
In this case, the result is not much different than `tsys`, because the $T_{sys}$ weights were already pretty uniform.
We also baseline subtract and plot the results.

In [ ]:
ta2 = pssb.timeaverage(weights=None)  
ta2.baseline(degree=1, remove=True, exclude=exclude_regions)
ta2.plot(ymin=-0.25, ymax=0.3)
print(f"final weights={ta2.weights}")
ta2.stats()

### 3. User-supplied weights
You can supply a numpy array of weights to apply.  For [`ScanBlock.timeaverage()`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock.timeaverage) the weights must have shape `(Nint,)` or `(Nint,nchan)` where `Nint` is the number of integrations in the [`ScanBlock`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock) and `nchan` is the number of channels in each scan.  

#### 3a.  Number of weights equal to number of integrations
We create a slightly silly example, that weights later integrations more.

In [ ]:
w = np.arange(1, pssb.nint+1, dtype=float)
ta3a = pssb.timeaverage(weights=w)
ta3a.baseline(degree=1, remove=True, exclude=exclude_regions)
ta3a.plot(ymin=-0.25, ymax=0.3)
print(f"final weights={ta3a.weights}")

#### 3b. Weights for each channel and integration
Supposed you had a channel-based $T_{sys}$ for each integration and wanted to calculate and apply system temperature weights. 
This can be accomplished by giving a weights array to [`ScanBlock.timeaverage()`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.scan.ScanBlock.timeaverage).

First we fake a `tsys` array of the correct shape.
Then we calculate system temperate weights using the mean exposure time and mean channel width of the scans.

In [ ]:
tsys = 30 + np.random.rand(pssb.nint, pssb.nchan)*15.0  # Fake system temperature array in the 30-45K range.
# The mean exposure and channel width.
dt = np.mean(np.mean(pssb.exposure))
df = np.mean(np.mean(pssb.delta_freq))

# Compute new weights using the inverse variance as given by the radiometer equation.
w = tsys_weight(dt, df, tsys)
print(f"{tsys=}")
print(f"Weights array shape: {w.shape}")

Now time average the data using the weights defined above, and repeat the baseline subtraction and plotting.

In [ ]:
ta3b = pssb.timeaverage(weights=w)
ta3b.baseline(degree=1, remove=True, exclude=exclude_regions)
ta3b.plot(ymin=-0.25, ymax=0.3)
print(f"final weights={ta3b.weights}")

Note the scan weights have been updated, and are now equal to the weights supplied to the `timeaverage` function, so their shape  is now (nint, nchan).

In [ ]:
print(pssb[0].weights.shape)
print(f"Are the Scan weights the same as those we defined? {np.all(w[0:11] == pssb[0].weights)}")

## Applying weights when averaging Spectrum
The method [`average_spectra`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.average_spectra) can compute weighted averages in 3 ways.
The first two are the usual `weights='tsys'` and `weights=None` options.
The third option, `weights='spectral'` will average the spectra using the values in each of their `weights` array.
Note that [`average_spectra`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.average_spectra) is the function used by [`Spectrum.average`](https://dysh.readthedocs.io/en/latest/reference/modules/dysh.spectra.html#dysh.spectra.spectrum.Spectrum.average) to average spectra.

In [ ]:
sp = average_spectra([ta1, ta2, ta3a, ta3b], weights='spectral')

In [ ]:
sp.plot(ymin=-0.25, ymax=0.3)
print(f"final weights={sp.weights}")